In [23]:
import sys
import os

sys.dont_write_bytecode = True

# 현재 작업 디렉토리 얻기
current_dir = os.getcwd()

# src 경로를 작업 디렉토리와 결합
my_path = os.path.join(current_dir, '../sootech')

# src 폴더를 Python 경로에 추가
sys.path.append(my_path)

# 이후 모듈 불러오기
from soo_classes import SooTech
import numpy as np
import pandas as pd


# Test Dataset 생성

In [24]:
F_data = pd.DataFrame(
    {
        'datetime': [
            "2024-01-01 00:00:00", "2024-01-01 00:00:10", "2024-01-01 00:00:20",
            "2024-01-01 00:00:30", "2024-01-01 00:00:40", "2024-01-01 00:00:50",
            "2024-01-01 00:01:00", "2024-01-01 00:01:10", "2024-01-01 00:01:20"
        ],
        'WS': [1, 10, 1, 10, 10, 10,  10, 10, 10],
        'WD': [0,  0, 0,  0, 90, 10, 350, 90, 45]
    }
)
A_data = pd.DataFrame(
    {
        'datetime': [
            "2024-01-01 00:00:00", "2024-01-01 00:00:10", "2024-01-01 00:00:20",
            "2024-01-01 00:00:30", "2024-01-01 00:00:40", "2024-01-01 00:00:50",
            "2024-01-01 00:01:00", "2024-01-01 00:01:10", "2024-01-01 00:01:20"
        ],
        'WS': [10, 1, 1, 10, 10,  10,  10, 15, np.nan],
        'WD': [ 0, 0, 0, 90,  0, 350,  10, 45, np.inf]
    }
)

In [25]:
soo = SooTech()

In [26]:

soo.resumetable(A_data)

데이터셋 형태: (9, 3)
Dataset shape: (9, 3)


,Feature,Data Type,Missing Values Count,Missing Values Percentage,Unique Values Count,Min Value,Max Value,First Value,Second Value,Third Value,Second to Last Value,Last Value
0,datetime,object,0,0.000000,9,2024-01-01 00:00:00,2024-01-01 00:01:20,2024-01-01 00:00:00,2024-01-01 00:00:10,2024-01-01 00:00:20,2024-01-01 00:01:10,2024-01-01 00:01:20
1,WS,float64,1,11.111111,3,1.0,15.0,10.0,1.0,1.0,15.0,NaN
2,WD,float64,0,0.000000,6,0.0,inf,0.0,0.0,0.0,45.0,inf


## seed_everything

In [27]:
soo.seed_everything(9234)

PyTorch가 설치되어 있지 않습니다. PyTorch 시드 설정은 건너뜁니다.


## trans_to_UV

In [28]:
uv_data = soo.trans_to_UV(ws=F_data.WS, wd=F_data.WD)
uv_data

,u,v
0,-0.000000,-1.000000e+00
1,-0.000000,-1.000000e+01
2,-0.000000,-1.000000e+00
3,-0.000000,-1.000000e+01
4,-10.000000,-6.123234e-16
5,-1.736482,-9.848078e+00
6,1.736482,-9.848078e+00
7,-10.000000,-6.123234e-16
8,-7.071068,-7.071068e+00


## trans_to_WSWD

In [29]:
soo.trans_to_WSWD(u=uv_data.u, v=uv_data.v)

,ws,wd
0,1.0,0.0
1,10.0,0.0
2,1.0,0.0
3,10.0,0.0
4,10.0,90.0
5,10.0,10.0
6,10.0,350.0
7,10.0,90.0
8,10.0,45.0


In [30]:
com_data = pd.merge(left=F_data, right=A_data, on="datetime", suffixes=["_F", "_A"])
com_data

,datetime,WS_F,WD_F,WS_A,WD_A
0,2024-01-01 00:00:00,1,0,10.0,0.0
1,2024-01-01 00:00:10,10,0,1.0,0.0
2,2024-01-01 00:00:20,1,0,1.0,0.0
3,2024-01-01 00:00:30,10,0,10.0,90.0
4,2024-01-01 00:00:40,10,90,10.0,0.0
5,2024-01-01 00:00:50,10,10,10.0,350.0
6,2024-01-01 00:01:00,10,350,10.0,10.0
7,2024-01-01 00:01:10,10,90,15.0,45.0
8,2024-01-01 00:01:20,10,45,NaN,inf


# statistic.py

In [31]:
windy = SooTech(com_data)

In [32]:
windy.me(com_data.WS_F, com_data.WS_A)

d:\GitHub\SooTech\test\../sootech\soo_statistic.py:24: UserWarning: 데이터에 NaN 또는 inf 값이 포함되어 있습니다.
  warnings.warn("데이터에 NaN 또는 inf 값이 포함되어 있습니다.")


np.float64(-0.625)

In [33]:
windy.mae(com_data.WS_F, com_data.WS_A)

np.float64(2.875)

In [34]:
windy.rmse(F_data.WS, A_data.WS)

d:\GitHub\SooTech\test\../sootech\soo_statistic.py:24: UserWarning: 데이터에 NaN 또는 inf 값이 포함되어 있습니다.
  warnings.warn("데이터에 NaN 또는 inf 값이 포함되어 있습니다.")


np.float64(4.834769901453429)

In [35]:
windy.d(F_data.WS, A_data.WS)

d:\GitHub\SooTech\test\../sootech\soo_statistic.py:24: UserWarning: 데이터에 NaN 또는 inf 값이 포함되어 있습니다.
  warnings.warn("데이터에 NaN 또는 inf 값이 포함되어 있습니다.")


np.float64(0.6182699668282725)

In [36]:
windy.cal_statistics(F_data.WS, A_data.WS)

d:\GitHub\SooTech\test\../sootech\soo_statistic.py:24: UserWarning: 데이터에 NaN 또는 inf 값이 포함되어 있습니다.
  warnings.warn("데이터에 NaN 또는 inf 값이 포함되어 있습니다.")
d:\GitHub\SooTech\test\../sootech\soo_statistic.py:24: UserWarning: 데이터에 NaN 또는 inf 값이 포함되어 있습니다.
  warnings.warn("데이터에 NaN 또는 inf 값이 포함되어 있습니다.")
d:\GitHub\SooTech\test\../sootech\soo_statistic.py:24: UserWarning: 데이터에 NaN 또는 inf 값이 포함되어 있습니다.
  warnings.warn("데이터에 NaN 또는 inf 값이 포함되어 있습니다.")


,F_mean,A_mean,ME,Pbias,MAE,RMSE,IOA,R,bcRMSE,BCPI,SBCPI
0,8.0,8.375,-0.625,-4.477612,2.875,4.83477,0.61827,NaN,4.794202,NaN,NaN


In [37]:
windy.raw_data = com_data.dropna()

In [38]:
windy.cal_statistics(windy.raw_data.WS_F, windy.raw_data.WS_A)

,F_mean,A_mean,ME,Pbias,MAE,RMSE,IOA,R,bcRMSE,BCPI,SBCPI
0,7.75,8.375,-0.625,-7.462687,2.875,4.83477,0.61827,0.364529,4.794202,21.271908,70.906361


In [39]:
windy.wd_diff_cal(windy.raw_data.WS_F, windy.raw_data.WD_F, windy.raw_data.WS_A, windy.raw_data.WD_A)

,Difference (degrees)
0,0.0
1,0.0
2,0.0
3,90.0
4,90.0
5,0.0
6,0.0
7,0.0


In [40]:
windy.raw_data

,datetime,WS_F,WD_F,WS_A,WD_A
0,2024-01-01 00:00:00,1,0,10.0,0.0
1,2024-01-01 00:00:10,10,0,1.0,0.0
2,2024-01-01 00:00:20,1,0,1.0,0.0
3,2024-01-01 00:00:30,10,0,10.0,90.0
4,2024-01-01 00:00:40,10,90,10.0,0.0
5,2024-01-01 00:00:50,10,10,10.0,350.0
6,2024-01-01 00:01:00,10,350,10.0,10.0
7,2024-01-01 00:01:10,10,90,15.0,45.0


In [41]:
print(windy.mean_wd(windy.raw_data.WS_A, windy.raw_data.WD_A, cal_method="mean"))
windy.mean_wd(com_data.WS_F, com_data.WD_F)

21.50379119889061


np.int64(0)